## 1. Text 데이터 다루기

## 멋진 단어사전 만들기

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()
print(mecab.morphs('자연어처리가너무재밌어서밥먹는것도가끔까먹어요'))

In [ ]:
#!pip uninstall --upgrade numpy
#!pip uninstall numpy

In [ ]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

%matplotlib inline

In [ ]:
import os
path_to_file = os.getenv('HOME')+'/aiffel/workspace/goingdeeper/sp_tokenizer/data/korean-english-park.train.ko'

with open(path_to_file, "r") as f:
    raw = f.read().splitlines()

print("Data Size:", len(raw))

print("Example:")
for sen in raw[0:100][::20]: print(">>", sen)

In [ ]:
min_len = 999
max_len = 0
sum_len = 0

for sen in raw:
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length

print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(raw))

sentence_length = np.zeros((max_len), dtype=int)

for sen in raw:
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

In [ ]:
def check_sentence_with_length(raw, length):
    count = 0
    
    for sen in raw:
        if len(sen) == length:
            print(sen)
            count += 1
            if count > 100: return

check_sentence_with_length(raw, 1)

In [ ]:
for idx, _sum in enumerate(sentence_length):
    # 문장의 수가 1500을 초과하는 문장 길이를 추출합니다.
    if _sum > 1500:
        print("Outlier Index:", idx+1)

In [ ]:
check_sentence_with_length(raw, 11)

In [2]:
min_len = 999
max_len = 0
sum_len = 0

cleaned_corpus = list(set(raw))  # set를 사용해서 중복을 제거합니다.
print("Data Size:", len(cleaned_corpus))

for sen in cleaned_corpus:
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length

print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(cleaned_corpus))

sentence_length = np.zeros((max_len), dtype=int)

for sen in cleaned_corpus:   # 중복이 제거된 코퍼스 기준
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

NameError: name 'raw' is not defined

In [ ]:
max_len = 150
min_len = 10

# 길이 조건에 맞는 문장만 선택합니다.
filtered_corpus = [s for s in cleaned_corpus if (len(s) < max_len) & (len(s) >= min_len)]

# 분포도를 다시 그려봅니다.
sentence_length = np.zeros((max_len), dtype=int)

for sen in filtered_corpus:
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

### 공백기반 토큰화

In [ ]:
def tokenize(corpus):  # corpus: Tokenized Sentence's List
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [ ]:
# 정제된 데이터 filtered_corpus를 공백 기반으로 토큰화하여 저장하는 코드를 직접 작성해 보세요.
split_corpus = []

for kor in filtered_corpus:
    # 코드를 작성하세요
    split_corpus.append(kor.split())

In [ ]:
split_tensor, split_tokenizer = tokenize(split_corpus)

print("Split Vocab Size:", len(split_tokenizer.index_word))

In [ ]:
for idx, word in enumerate(split_tokenizer.word_index):
    print(idx, ":", word)

    if idx > 10: break

### 형태소 기반 토큰화

In [ ]:
# 위에서 사용한 코드를 활용해 MeCab 단어 사전을 만들어보세요. 
# Hint : mecab.morphs()를 사용해서 형태소분석을 합니다.
def mecab_split(sentence):
    # 코드를 작성하세요
    split_kor = mecab.morphs(sentence)
    #print(split_kor[0])
    return split_kor

mecab_corpus = []

for kor in filtered_corpus:
    mecab_corpus.append(mecab_split(kor))


In [ ]:
mecab_tensor, mecab_tokenizer = tokenize(mecab_corpus)

print("MeCab Vocab Size:", len(mecab_tokenizer.index_word))

In [ ]:
# Case 1 : tokenizer.sequences_to_texts()

# 여기에 코드를 작성하세요.
texts = mecab_tokenizer.sequences_to_texts([mecab_tensor[100]])
print(texts[0])


In [ ]:
# Case 2 : tokenizer.index_word

# 여기에 코드를 작성하세요.
sentence = ""

for w in mecab_tensor[100]:
    if w == 0: continue
    sentence += mecab_tokenizer.index_word[w] + " "

print(sentence)

### 프로젝트: SentencePiece 사용하기

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib as plt

import konlpy

print(tf.__version__)
print(np.__version__)
print(plt.__version__)
print(konlpy.__version__)

### sentence piece 불러오기

In [ ]:
import sentencepiece as spm
import os
#temp_file = os.getenv('HOME')+'/aiffel/workspace/goingdeeper/sp_tokenizer/data/korean-english-park.train.ko.temp'
temp_file = os.getenv('HOME')+'/aiffel/workspace/goingdeeper/sp_tokenizer/data/korean_spm_bpe_10k.model'



korean_spm_bpe_10k.model

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in filtered_corpus:   # 이전에 나왔던 정제했던 corpus를 활용해서 진행해야 합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
)

In [ ]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds('아버지가방에들어가신다.')
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces('아버지가방에들어가신다.',1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

In [23]:
def sp_tokenize(s, corpus): 

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({word:idx})
        index_word.update({idx:word})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

### 네이버 영화리뷰 감정 분석 문제에 SentencePiece 적용해 보기

In [ ]:
import pandas as pd

train_data = pd.read_table('~/aiffel/workspace/goingdeeper/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/workspace/goingdeeper/sentiment_classification/ratings_test.txt')

train_data.head()


In [ ]:
test_data.head()

In [ ]:
print('Train dataset size :',len(train_data))
print('Test dataset size :',len(test_data)) 

In [ ]:
df = pd.concat([train_data, test_data])
document_label = df.loc[:, ['document', 'label']]

document_label

In [ ]:
len(df)

In [ ]:
document_label.drop_duplicates(subset=['document'], inplace=True)
document_label = train_data.dropna(how = 'any')   

In [ ]:
len(document_label)

In [ ]:
import matplotlib.pyplot as plt

min_len = 999
max_len = 0
sum_len = 0

for sen in document_label["document"]:
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length
    
print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(raw))



In [ ]:
sentence_length = np.zeros((max_len), dtype=int)
 
 
for sen in document_label["document"]:
    sentence_length[len(sen)-1] += 1

In [ ]:
def check_sentence_with_length(raw, length):
    count = 0
    
    for sen in raw:
        if len(sen) == length:
            print(sen)
            count += 1
            if count > 20: return

check_sentence_with_length(document_label["document"], 1)

In [ ]:
for idx, _sum in enumerate(sentence_length):
    # 문장의 수가 1500을 초과하는 문장 길이를 추출합니다.
    if _sum > 1500:
        print("Outlier Index:", idx+1)

In [ ]:
check_sentence_with_length(document_label["document"], 46)

min_len = 999
max_len = 0
sum_len = 0

cleaned_corpus = list(set(df["document"]))  # set를 사용해서 중복을 제거합니다.
print("Data Size:", len(cleaned_corpus))
df[0]


In [ ]:
min_len = 999
max_len = 0
sum_len = 0

#cleaned_corpus = list(set(df["document"]))  # set를 사용해서 중복을 제거합니다.
#print("Data Size:", len(cleaned_corpus))

cleaned_corpus = document_label

for sen in cleaned_corpus["document"]:
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length

print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(cleaned_corpus["document"]))

sentence_length = np.zeros((max_len), dtype=int)

for sen in cleaned_corpus["document"]:   # 중복이 제거된 코퍼스 기준
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

In [ ]:
filtered_corpus[0]

In [ ]:
max_len = 100#150
min_len = 10

# 길이 조건에 맞는 문장만 선택합니다.
#filtered_corpus = [s for s in cleaned_corpus if (len(s) < max_len) & (len(s) >= min_len)]
filtered_corpus = cleaned_corpus[cleaned_corpus['document'].str.len().between(min_len, max_len)]

# 분포도를 다시 그려봅니다.
sentence_length = np.zeros((max_len), dtype=int)

for sen in filtered_corpus['document']:
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

### 모델 설계 및 학습

In [72]:
import tensorflow as tf
from tensorflow import keras

def get_model():
    
    vocab_size = 10000  # 어휘 사전의 크기입니다.
    word_vector_dim = 200  # 워드 벡터의 차원수

    # model 설계
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
    model.add(keras.layers.LSTM(word_vector_dim))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경가능)
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Droupout(0.5))
    model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.summary()
   
    return model

### SentencePiece 기반 토큰화

In [ ]:
#temp_file = os.getenv('HOME')+'/aiffel/workspace/goingdeeper/sp_tokenizer/data/korean-english-park.train.ko.temp'
temp_file = os.getenv('HOME')+'/aiffel/workspace/goingdeeper/sp_tokenizer/data/korean_spm_bpe_10k.vocab'




vocab_size = 10000

with open(temp_file, 'w') as f:
    for row in filtered_corpus['document']:   # 이전 스텝에서 정제했던 corpus를 활용합니다.
        f.write(str(row) + '\n')


spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size))    

In [ ]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus['document']:
        tokensIDs = s.EncodeAsIds(sen)
        #print(tokensIDs)
        #print(s.SampleEncodeAsPieces(sen),1, 0.0)
        #print(s.DecodeIds(tokensIDs))
        
        tensor.append(s.EncodeAsIds(sen))
    #print(tensor)

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word


In [ ]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds('아버지가방에들어가신다.')
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces('아버지가방에들어가신다.',1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

x_train, word_index, index_word = sp_tokenize(s, filtered_corpus)
x_train.shape

In [ ]:
x_train[0]

In [ ]:
from sklearn.model_selection import train_test_split

y_train = filtered_corpus['label']

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)


In [ ]:
len(x_train), len(y_train)

In [ ]:
model_sp = get_model()
model_sp.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

### 공백 기반 토큰화

In [44]:
def tokenize(corpus):  # corpus: Tokenized Sentence's List
    
    split_corpus = []

    for kor in filtered_corpus['document']:
        split_corpus.append(kor.split())

    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return split_corpus,tokenizer 



x_train, tokenizer = tokenize(split_corpus)


In [45]:
y_train = filtered_corpus['label']

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)


In [46]:
len(x_train), len(y_train)

(102448, 102448)

In [ ]:
model_split = get_model()
model_split.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

### 형태소 기반 토큰화

In [47]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
m = Mecab()

def get_mecab_tokenize(data):
    mecab_corpus = []
    
    for sentence in data['document']:
        mecab_corpus.append(m.morphs(sentence))

    tokenizer.fit_on_texts(mecab_corpus)
    mecab_tensor = tokenizer.texts_to_sequences(mecab_corpus)
    mecab_tensor = tf.keras.preprocessing.sequence.pad_sequences(mecab_tensor, padding='post', maxlen=max_len)
    return mecab_tensor


x_train = get_mecab_tokenize(filtered_corpus)

In [48]:
len(x_train), len(filtered_corpus)

(128060, 128060)

In [49]:
y_train = filtered_corpus['label']

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)


In [50]:
len(x_train), len(y_train)

(102448, 102448)

In [ ]:
model_mecab = get_model()
model_mecab.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))